# List of Satellite with respective height and time of culmination pass

In [2]:
from skyfield.api import Topos, load, EarthSatellite
from datetime import datetime
import requests


# Function to download TLE data from CelesTrak
def download_tle():
    url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    tle_lines = response.text.strip().splitlines()
    return tle_lines

# Load satellites using EarthSatellite class
def load_satellites(tle_lines):
    satellites = []
    for i in range(0, len(tle_lines), 3):
        name = tle_lines[i].strip()
        if "STARLINK" not in name.upper():  # Exclude Starlink satellites
            line1 = tle_lines[i + 1].strip()
            line2 = tle_lines[i + 2].strip()
            satellite = EarthSatellite(line1, line2, name, load.timescale())
            satellites.append(satellite)
    return satellites

# Function to predict satellite culmination passes for a given location and time range
def predict_passes(satellites, location, start_time, end_time):
    ts = load.timescale()
    observer = Topos(latitude_degrees=location[0], longitude_degrees=location[1])

    results = []  # List to store results for sorting
    print(f"Predicting satellite culminations for location: {location}")
    for satellite in satellites:
        t, events = satellite.find_events(observer, start_time, end_time, altitude_degrees=30.0)
        for ti, event in zip(t, events):
            if event == 1:  # 1 corresponds to 'culminate'
                # Calculate satellite altitude during culmination
                difference = satellite - observer
                topocentric = difference.at(ti)
                altitude = topocentric.distance().km  # Altitude in kilometers
                
                # Append result as a tuple (satellite name, time, altitude)
                results.append((satellite.name, ti.utc_strftime('%Y-%m-%d %H:%M:%S'), altitude))

    # Sort results by altitude
    results.sort(key=lambda x: x[2], reverse=True)  # Sort by altitude in descending order

    # Display sorted results
    for name, time, altitude in results:
        print(f"{name} culminate at {time} UTC, altitude: {altitude:.2f} km")

# Main function
if __name__ == "__main__":
    # Download and load TLE data
    tle_lines = download_tle()
    satellites = load_satellites(tle_lines)

    # Set location for Zurich, Switzerland (latitude, longitude)
    location = (47.3769, 8.5417)

    # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)
    ts = load.timescale()
    start_time = ts.utc(2025, 1, 7, 17, 15, 0)  # 8 Jan 2025, 01:00 UTC
    end_time = ts.utc(2025, 1, 7, 17, 40, 0)    # 8 Jan 2025, 02:00 UTC

    # Predict satellite culmination passes
    predict_passes(satellites, location, start_time, end_time)

Predicting satellite culminations for location: (47.3769, 8.5417)
THOR 5 culminate at 2025-01-07 17:38:50 UTC, altitude: 38182.09 km
BEIDOU-2 IGSO-3 culminate at 2025-01-07 17:25:50 UTC, altitude: 38176.88 km
ARKTIKA-M 2 culminate at 2025-01-07 17:20:42 UTC, altitude: 28706.96 km
GSAT0203 (GALILEO 7) culminate at 2025-01-07 17:20:15 UTC, altitude: 23443.79 km
NAVSTAR 78 (USA 293) culminate at 2025-01-07 17:19:28 UTC, altitude: 20447.65 km
NAVSTAR 73 (USA 260) culminate at 2025-01-07 17:28:19 UTC, altitude: 20349.43 km
COSMOS 2522 [GLONASS-M] culminate at 2025-01-07 17:17:33 UTC, altitude: 19275.52 km
COSMOS 2545 [GLONASS-M] culminate at 2025-01-07 17:35:57 UTC, altitude: 19265.84 km
LAGEOS 1 culminate at 2025-01-07 17:30:05 UTC, altitude: 7197.81 km
LAGEOS 2 culminate at 2025-01-07 17:25:29 UTC, altitude: 6001.22 km
EXPRESS-MD2 culminate at 2025-01-07 17:21:28 UTC, altitude: 2586.07 km
GLOBALSTAR M085 culminate at 2025-01-07 17:18:40 UTC, altitude: 2238.59 km
COSMOS 2386 culminate at 2

In [10]:
import csv
from skyfield.api import Topos, load, EarthSatellite
from datetime import datetime
import requests

# Function to download TLE data from CelesTrak
def download_tle():
    url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    tle_lines = response.text.strip().splitlines()
    return tle_lines

# Load satellites using EarthSatellite class
def load_satellites(tle_lines):
    satellites = []
    for i in range(0, len(tle_lines), 3):
        name = tle_lines[i].strip()
        if "STARLINK" not in name.upper():  # Exclude Starlink satellites
            line1 = tle_lines[i + 1].strip()
            line2 = tle_lines[i + 2].strip()
            satellite = EarthSatellite(line1, line2, name, load.timescale())
            satellites.append(satellite)
    return satellites

# Function to predict satellite culmination passes for a given location and time range
def predict_passes(satellites, location, start_time, end_time, output_csv="satellite_passes.csv"):
    ts = load.timescale()
    observer = Topos(latitude_degrees=location[0], longitude_degrees=location[1])

    results = []  # List to store results for sorting
    print(f"Predicting satellite culminations for location: {location}")
    for satellite in satellites:
        t, events = satellite.find_events(observer, start_time, end_time, altitude_degrees=30.0)
        for ti, event in zip(t, events):
            if event == 1:  # 1 corresponds to 'culminate'
                # Calculate satellite altitude during culmination
                difference = satellite - observer
                topocentric = difference.at(ti)
                altitude = topocentric.distance().km  # Altitude in kilometers
                
                # Append result as a tuple (satellite name, time, altitude)
                results.append((satellite.name, ti.utc_strftime('%Y-%m-%d %H:%M:%S'), altitude))

    # Sort results by altitude
    results.sort(key=lambda x: x[2], reverse=True)  # Sort by altitude in descending order

    # Write sorted results to CSV file
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Satellite Name", "Culmination Time (UTC)", "Altitude (km)"])  # Header
        writer.writerows(results)

    # Display sorted results
    for name, time, altitude in results:
        print(f"{name} culminate at {time} UTC, altitude: {altitude:.2f} km")

# Main function
if __name__ == "__main__":
    # Download and load TLE data
    tle_lines = download_tle()
    satellites = load_satellites(tle_lines)

    # Set location for Zurich, Switzerland (latitude, longitude)
    location = (47.3769, 8.5417)

            # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)
    #1st
    ts = load.timescale()
    start_time = ts.utc(2025, 1, 7, 17, 00, 0)  # 8 Jan 2025, 01:00 UTC
    end_time = ts.utc(2025, 1, 7, 18, 00, 0)    # 8 Jan 2025, 02:00 UTC

    # Predict satellite culmination passes and export to CSV
    predict_passes(satellites, location, start_time, end_time)
"""
    # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)
    #1st
    ts = load.timescale()
    start_time = ts.utc(2025, 1, 7, 18, 00, 0)  # 8 Jan 2025, 01:00 UTC
    end_time = ts.utc(2025, 1, 7, 21, 00, 0)    # 8 Jan 2025, 02:00 UTC

    # Predict satellite culmination passes and export to CSV
    predict_passes(satellites, location, start_time, end_time)

    # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)
    #1st
    ts = load.timescale()
    start_time = ts.utc(2025, 1, 7, 21, 00, 0)  # 8 Jan 2025, 01:00 UTC
    end_time = ts.utc(2025, 1, 8, 00, 00, 0)    # 8 Jan 2025, 02:00 UTC

    # Predict satellite culmination passes and export to CSV
    predict_passes(satellites, location, start_time, end_time)

        # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)
    #1st
    ts = load.timescale()
    start_time = ts.utc(2025, 1, 8, 00, 00, 0)  # 8 Jan 2025, 01:00 UTC
    end_time = ts.utc(2025, 1, 8, 3, 00, 0)    # 8 Jan 2025, 02:00 UTC
    # Predict satellite culmination passes and export to CSV
    predict_passes(satellites, location, start_time, end_time)
"""





Predicting satellite culminations for location: (47.3769, 8.5417)
2024-252A culminate at 2025-01-07 17:57:53 UTC, altitude: 47973.54 km
AMOS-3 culminate at 2025-01-07 17:46:22 UTC, altitude: 38238.70 km
THOR 5 culminate at 2025-01-07 17:38:50 UTC, altitude: 38182.09 km
BEIDOU-2 IGSO-3 culminate at 2025-01-07 17:25:50 UTC, altitude: 38176.88 km
BEIDOU-2 IGSO-6 culminate at 2025-01-07 17:46:52 UTC, altitude: 38091.12 km
ARKTIKA-M 2 culminate at 2025-01-07 17:20:42 UTC, altitude: 28707.12 km
GSAT0203 (GALILEO 7) culminate at 2025-01-07 17:20:15 UTC, altitude: 23443.79 km
BEIDOU-3 M8 culminate at 2025-01-07 17:12:27 UTC, altitude: 21547.38 km
NAVSTAR 78 (USA 293) culminate at 2025-01-07 17:19:28 UTC, altitude: 20447.65 km
NAVSTAR 73 (USA 260) culminate at 2025-01-07 17:28:19 UTC, altitude: 20349.43 km
NAVSTAR 63 (USA 203) culminate at 2025-01-07 17:01:30 UTC, altitude: 20025.15 km
COSMOS 2522 [GLONASS-M] culminate at 2025-01-07 17:17:32 UTC, altitude: 19275.53 km
COSMOS 2545 [GLONASS-M] cu

'\n    # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)\n    #1st\n    ts = load.timescale()\n    start_time = ts.utc(2025, 1, 7, 18, 00, 0)  # 8 Jan 2025, 01:00 UTC\n    end_time = ts.utc(2025, 1, 7, 21, 00, 0)    # 8 Jan 2025, 02:00 UTC\n\n    # Predict satellite culmination passes and export to CSV\n    predict_passes(satellites, location, start_time, end_time)\n\n    # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)\n    #1st\n    ts = load.timescale()\n    start_time = ts.utc(2025, 1, 7, 21, 00, 0)  # 8 Jan 2025, 01:00 UTC\n    end_time = ts.utc(2025, 1, 8, 00, 00, 0)    # 8 Jan 2025, 02:00 UTC\n\n    # Predict satellite culmination passes and export to CSV\n    predict_passes(satellites, location, start_time, end_time)\n\n        # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)\n    #1st\n    ts = load.timescale()\n    start_time = ts.utc(2025, 1, 8, 00, 00, 0)  # 8 Jan 2025, 01:00 UTC\n    end_tim